In [2]:
import pandas as pd

In [3]:
bakery_sales = pd.read_csv('Bakery_sales.csv')
Coffee_sales = pd.read_csv('Coffee_Shop_sales.csv')

In [4]:
#clean the unit_price column by removing the € sign and converting the column to float
bakery_sales["unit_price"] = (
    bakery_sales["unit_price"]
    .str.replace("€", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace('"', "", regex=False)
    .str.strip()
    .astype(float)
)
#rename the columns to match the target schema
bakery_sales = bakery_sales.rename(
    columns={
        "ticket_number": "transaction_id",
        "date": "transaction_date",
        "time": "transaction_time",
        "Quantity": "transaction_qty",
        "article": "product_detail",
    }
)
#add the missing columns
bakery_sales["store_id"] = None
bakery_sales["store_location"] = None
bakery_sales["product_id"] = None
bakery_sales["product_category"] = "Bakery"
bakery_sales["product_type"] = None
#reorder the columns
bakery_sales = bakery_sales[
    [
        "transaction_id",
        "transaction_date",
        "transaction_time",
        "transaction_qty",
        "store_id",
        "store_location",
        "product_id",
        "unit_price",
        "product_category",
        "product_type",
        "product_detail",
    ]
]


In [5]:
bakery_sales.to_csv("bakery_sales_new.csv", index=False)

In [6]:
unique_categories = Coffee_sales['product_category'].unique()
print(unique_categories)

['Coffee' 'Tea' 'Drinking Chocolate' 'Bakery' 'Flavours' 'Loose Tea'
 'Coffee beans' 'Packaged Chocolate' 'Branded']


In [7]:
# filter the Coffee_sales DataFrame for Bakery category and list unique product types and details
bakery_items = Coffee_sales[Coffee_sales['product_category'] == 'Bakery'][['product_type', 'product_detail']].drop_duplicates()
print(bakery_items.nunique())


product_type       3
product_detail    11
dtype: int64


In [ ]:
# read the new CSV file and perform the same analysis on it
bakery_sales_new = pd.read_csv("bakery_sales_new.csv")
bakery_items_new = bakery_sales_new[bakery_sales_new['product_category'] == 'Bakery'][['product_detail']].drop_duplicates()
print(bakery_items_new['product_detail'].unique())


In [ ]:
def map_article_to_category(article):
    normalized = article.strip().upper()
    
    if "CAFE" in normalized:
        return "Coffee", "Coffee", article
    if normalized == "THE":
        return "Tea", "Tea", article
    
    category = "Bakery"
    
    breads = ['BAGUETTE', 'PAIN', 'BOULE', 'PAIN DE MIE', 'SEIGLE', 'VIK BREAD', 'PAIN BANETTE', 'PAIN NOIR']
    viennoiseries = ['PAIN AU CHOCOLAT', 'CROISSANT', 'KOUIGN AMANN', 'BRIOCHE', 'CHAUSSON', 'VIENNOISERIE']
    patisseries = ['TARTE', 'ECLAIR', 'FRAISIER', 'ST HONORE', 'MACARON', 'ENTREMETS', 'PARIS BREST', 'ROYAL', 'TROPEZIENNE',
                   'FINANCIER', 'CRUMBLE', 'MILLES FEUILLES', 'RELIGIEUSE', 'MERINGUE']
    sandwiches = ['SANDWICH', 'SAND JB', 'FORMULE SANDWICH']
    beverages = ['BOISSON', 'EAU']
    meals = ['PLAT', 'TRAITEUR', 'PLATEAU', 'FORMULE PLAT PREPARE', 'PLATPREPARE']
    confections = ['CONFISERIE', 'SUCETTE', 'CHOCOLAT', 'MERINGUE', 'SABLE', 'BONBON']
    
    if any(word in normalized for word in breads):
        prod_type = "Bread"
    elif any(word in normalized for word in viennoiseries):
        prod_type = "Viennoiserie"
    elif any(word in normalized for word in patisseries):
        prod_type = "Pâtisserie"
    elif any(word in normalized for word in sandwiches):
        prod_type = "Sandwich"
    elif any(word in normalized for word in meals):
        prod_type = "Meal/Traiteur"
    elif any(word in normalized for word in beverages):
        prod_type = "Beverage"
    elif any(word in normalized for word in confections):
        prod_type = "Confectionery"
    else:
        prod_type = "Other"
        
    return category, prod_type, article

bakery_sales_new[['product_category', 'product_type', 'product_detail']] = bakery_sales_new['product_detail'].apply(
    lambda x: pd.Series(map_article_to_category(x))
)

bakery_sales_new.to_csv("bakery_sales_merged.csv", index=False)


NameError: name 'bakery_sales_new' is not defined